## Updating and Deleting Rows with Core

So far we've covered `Insert`, so that we can get some data into our database, and then spent a lot of time on `Select` which _handles the broad range of usage patterns_ used for __retrieving data from the database__. In this section we will cover the `Update` and `Delete` constructs, which are used to __modify existing rows__ as well as __delete existing rows__. This section will cover these constructs from a _Core-centric perspective_.

> `ORM Readers` - As was the case mentioned at `Inserting Rows with Core`, the `Update` and `Delete` operations when used _with the ORM_ are usually __invoked internally from the Session object__ as part of the unit of work process.
>
> However, __unlike Insert__, the `Update` and `Delete` constructs __can also be used directly with the ORM__, using a pattern known as __`"ORM-enabled update and delete"`__; for this reason, familiarity with these constructs is useful for ORM use. Both styles of use are discussed in the sections `Updating ORM Objects` and `Deleting ORM Objects`.

In [17]:
from sqlalchemy import (
    MetaData, Table, Column, Integer, String, ForeignKey,
    create_engine, select, insert, update, delete, bindparam,
)
from sqlalchemy.orm import registry, relationship
from sqlalchemy.dialects import mysql

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
metadata_obj = MetaData()

In [3]:
user_table = Table(
    "user_account", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

address_table = Table(
    "address", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False)
)

metadata_obj.create_all(engine)

2022-10-05 17:36:53,412 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-05 17:36:53,415 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-10-05 17:36:53,416 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-05 17:36:53,419 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-10-05 17:36:53,420 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-05 17:36:53,423 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-10-05 17:36:53,425 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-05 17:36:53,427 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-10-05 17:36:53,429 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-05 17:36:53,432 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-10-05 17:36:53,433 INFO sqlalchemy.engine.Engine [no key 0.00143s] ()
2022-10-05 17:36:53,436 INFO sqlalchemy.engine.Engine 
C

In [4]:
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"}
        ]
    )
    conn.commit()

2022-10-05 17:36:53,582 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-05 17:36:53,584 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-10-05 17:36:53,585 INFO sqlalchemy.engine.Engine [generated in 0.00287s] ('spongebob', 'Spongebob Squarepants')
2022-10-05 17:36:53,587 INFO sqlalchemy.engine.Engine COMMIT
2022-10-05 17:36:53,589 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-05 17:36:53,591 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-10-05 17:36:53,592 INFO sqlalchemy.engine.Engine [generated in 0.00345s] (('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-10-05 17:36:53,596 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
scalar_subq = (
    select(user_table.c.id).
    where(user_table.c.name == bindparam("username")).
    scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {"username": 'spongebob', "email_address": "spongebob@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@squirrelpower.org"},
        ]
    )
    conn.commit()

2022-10-05 17:36:53,775 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-05 17:36:53,778 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2022-10-05 17:36:53,780 INFO sqlalchemy.engine.Engine [generated in 0.00473s] (('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org'))
2022-10-05 17:36:53,782 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
mapped_registry = registry()
Base = mapped_registry.generate_base()

In [7]:
class User(Base):
    __tablename__ = "user_account"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    addresses = relationship("Address", back_populates="user")
    
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

In [8]:
class Address(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"))
    
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [9]:
mapped_registry.metadata.create_all(engine)

2022-10-05 17:36:54,476 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-05 17:36:54,479 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-10-05 17:36:54,480 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-05 17:36:54,482 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-10-05 17:36:54,483 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-05 17:36:54,485 INFO sqlalchemy.engine.Engine COMMIT


#### The `update()` SQL Expression Construct

The `update()` function generates a _new instance of Update_ which represents an `UPDATE` statement in SQL, that will __update existing data in a table__. Like the `insert()` construct, there is a `"traditional"` form of `update()`, which __emits `UPDATE` against a single table__ at a time and __`does not` return any rows__. However _some backends_ support an `UPDATE` statement that __may modify multiple tables at once__, and the `UPDATE` statement also supports `RETURNING` such that _columns contained in matched rows may be returned in the result set_.

In [10]:
stmt = (
    update(user_table).where(user_table.c.name == "patrick").
    values(fullname="Patrick the Star")
)
print(stmt)

UPDATE user_account SET fullname=:fullname WHERE user_account.name = :name_1


The `Update.values()` method __controls the contents of the SET elements__ of the `UPDATE` statement. This is the __same method shared by the `Insert` construct__. Parameters can normally be _passed using the column names as keyword arguments_. `UPDATE` _supports all the major SQL forms_ of `UPDATE`, __including updates against expressions__, where we can make use of `Column` expressions.

In [11]:
stmt = update(user_table).values(fullname="Username: " + user_table.c.name)
print(stmt)

UPDATE user_account SET fullname=(:name_1 || user_account.name)


To support `UPDATE` in an __`"executemany"`__ context, where many parameter sets will be invoked against the same statement, the `bindparam()` construct may be used to __set up bound parameters__; these __replace the places__ that _literal values would normally go_.

In [12]:
stmt = (
    update(user_table).
    where(user_table.c.name == bindparam("oldname")).
    values(name=bindparam("newname"))
)

with engine.connect() as conn:
    conn.execute(
        stmt,
        [
            {'oldname':'jack', 'newname':'ed'},
            {'oldname':'wendy', 'newname':'mary'},
            {'oldname':'jim', 'newname':'jake'},
        ]
    )

2022-10-05 17:36:54,981 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-05 17:36:54,983 INFO sqlalchemy.engine.Engine UPDATE user_account SET name=? WHERE user_account.name = ?
2022-10-05 17:36:54,984 INFO sqlalchemy.engine.Engine [generated in 0.00288s] (('ed', 'jack'), ('mary', 'wendy'), ('jake', 'jim'))
2022-10-05 17:36:54,986 INFO sqlalchemy.engine.Engine ROLLBACK


##### Correlated Updates

An `UPDATE` statement can _make use of rows in other tables_ by using a __correlated subquery__. A `subquery` may be used _anywhere a column expression might be placed_.

In [13]:
scalar_subq = (
    select(address_table.c.email_address).
    where(address_table.c.user_id == user_table.c.id).
    order_by(address_table.c.id).
    limit(1).scalar_subquery()
)

update_stmt = update(user_table).values(fullname=scalar_subq)
print(update_stmt)

UPDATE user_account SET fullname=(SELECT address.email_address 
FROM address 
WHERE address.user_id = user_account.id ORDER BY address.id
 LIMIT :param_1)


##### UPDATE ... FROM

_Some databases_ such as `PostgreSQL` and `MySQL` support a syntax `"UPDATE FROM"` where _additional tables may be stated directly_ in a __special FROM clause__. This syntax will be __generated implicitly__ when additional tables are located in the `WHERE` clause of the statement.

In [14]:
update_stmt = (
    update(user_table).
    where(user_table.c.id == address_table.c.user_id).
    where(address_table.c.email_address == "patrick@aol.com").
    values(fullname="Pat")
)
print(update_stmt)

UPDATE user_account SET fullname=:fullname FROM address WHERE user_account.id = address.user_id AND address.email_address = :email_address_1


There is also a `MySQL` specific syntax that __can `UPDATE` multiple tables__. This __requires__ we refer to `Table` objects in the `VALUES` clause in order to _refer to additional tables_.

In [15]:
update_stmt = (
    update(user_table).
    where(user_table.c.id == address_table.c.user_id).
    where(address_table.c.email_address == "patrick@aol.com").
    values(
        {
            user_table.c.fullname: "Pat",
            address_table.c.email_address: "pat@aol.com",
        }
    )
)
print(update_stmt.compile(dialect=mysql.dialect()))

UPDATE user_account, address SET address.email_address=%s, user_account.fullname=%s WHERE user_account.id = address.user_id AND address.email_address = %s


##### Parameter Ordered Updates

Another _`MySQL`-only behavior_ is that the _order of parameters_ in the `SET clause` of an `UPDATE` actually __impacts the evaluation of each expression__. For this use case, the `Update.ordered_values()` method accepts a _sequence of tuples_ so that this __order may be controlled__.

In [16]:
update_stmt = (
    update(user_table).
    ordered_values(
        (user_table.c.name, "patrick"),
        (user_table.c.fullname, user_table.c.name + "surname")
    )
)
print(update_stmt)

UPDATE user_account SET name=:name, fullname=(user_account.name || :name_1)


While `Python dictionaries` are __guaranteed to be insert ordered__ as of Python 3.7, the `Update.ordered_values()` method still _provides an additional measure of clarity of intent_ when it is __essential__ that the `SET clause` of a `MySQL` `UPDATE` statement proceed in a specific way.

#### The `delete()` SQL Expression Construct

The `delete()` function _generates a new instance of Delete_ which represents a `DELETE` statement in SQL, that will __delete rows from a table__. The `delete()` statement from an _API perspective_ is __very similar__ to that of the `update()` construct, traditionally _returning no rows_ but __allowing for a RETURNING variant__ on _some database backends_.

In [18]:
stmt = delete(user_table).where(user_table.c.name == "patrick")
print(stmt)

DELETE FROM user_account WHERE user_account.name = :name_1


##### Multiple Table Deletes

Like `Update`, `Delete` supports the use of __correlated subqueries__ in the `WHERE` clause as well as _backend-specific_ __multiple table syntaxes__, such as `DELETE FROM...USING` on `MySQL`.

In [19]:
delete_stmt = (
    delete(user_table).
    where(user_table.c.id == address_table.c.user_id).
    where(address_table.c.email_address == "patrick@aol.com")
)
print(delete_stmt)

DELETE FROM user_account , address WHERE user_account.id = address.user_id AND address.email_address = :email_address_1


#### Getting Affected Row Count from UPDATE, DELETE

Both `Update` and `Delete` _support_ the ability to __return the number of rows matched__ after the statement proceeds, for _statements_ that are _invoked using Core_ `Connection`, i.e. `Connection.execute()`. Per the caveats mentioned below, this value is available from the `CursorResult.rowcount` attribute.

In [20]:
with engine.begin() as conn:
    result = conn.execute(
        update(user_table).
        values(fullname="Patrick McStar").
        where(user_table.c.name == "patrick")
    )
    print(result.rowcount)

2022-10-05 18:14:48,153 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-05 18:14:48,156 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.name = ?
2022-10-05 18:14:48,157 INFO sqlalchemy.engine.Engine [generated in 0.00136s] ('Patrick McStar', 'patrick')
1
2022-10-05 18:14:48,160 INFO sqlalchemy.engine.Engine COMMIT


> ##### Tips
>
> The `CursorResult` class is a _subclass of Result_ which _contains additional attributes_ that are specific to the `DBAPI cursor object`. _An instance of this subclass is returned_ when a statement is _invoked_ via the `Connection.execute()` method. When using the `ORM`, the `Session.execute()` method returns an object of this type for all `INSERT`, `UPDATE`, and `DELETE` statements.

Facts about `CursorResult.rowcount`:

* The _value returned_ is the __number of rows matched__ by the `WHERE` clause of the statement. It __does not matter__ _if the row were_ __actually modified or not__.

* `CursorResult.rowcount` is __not necessarily available__ for an `UPDATE` or `DELETE` statement that uses `RETURNING`.

* For an `executemany` execution, `CursorResult.rowcount` __may not be available__ either, which _depends highly on the DBAPI module_ in use as well as _configured options_. The attribute `CursorResult.supports_sane_multi_rowcount` _indicates if this value_ will be __available__ _for the current backend_ in use.

* _Some drivers_, particularly _third party dialects_ for __`non-relational databases`, may not support__ `CursorResult.rowcount` at all. The `CursorResult.supports_sane_rowcount` will indicate this.

* `"rowcount"` is used by the `ORM` unit of work process to _validate_ that an `UPDATE` or `DELETE` statement __matched the expected number of rows__, and is also _essential_ for the __ORM versioning feature__ documented at `Configuring a Version Counter`.